In [1]:
# Importing necessary packages
import pandas as pd
import numpy as np
import statsmodels.api as sm

**If using Stargazer to export LaTeX code:** \
!pip install Stargazer \
from stargazer.stargazer import Stargazer

In [2]:
data = pd.read_csv('factordata.csv')

# Multiplying by Indicators
data['IndGMB_U'] = data['GMB_U'] * data['Method_Estimated']
data['IndGMB_S'] = data['GMB_S'] * data['Method_Estimated']
data['IndMkt-RF'] = data['Mkt-RF'] * data['Method_Estimated']
data['IndSMB'] = data['SMB'] * data['Method_Estimated']
data['IndHML'] = data['HML'] * data['Method_Estimated']
data['IndRMW'] = data['RMW'] * data['Method_Estimated']
data['IndCMA'] = data['CMA'] * data['Method_Estimated']

print(data)

           cusip  year  Month  monthyear  Mkt-RF   SMB   HML    RF   RMW  \
0       46603210  2015      1     201501   -3.11 -0.92 -3.59  0.00  1.61   
1       46603210  2015      2     201502    6.13  0.32 -1.86  0.00 -1.12   
2       46603210  2015      3     201503   -1.12  3.07 -0.38  0.00  0.09   
3       46603210  2015      4     201504    0.59 -3.09  1.82  0.00  0.06   
4       46603210  2015      5     201505    1.36  0.84 -1.15  0.00 -1.80   
...          ...   ...    ...        ...     ...   ...   ...   ...   ...   
295824  12503M10  2023      8     202308   -2.39 -3.68 -1.08  0.45  3.42   
295825  12503M10  2023      9     202309   -5.24 -1.79  1.45  0.43  1.85   
295826  12503M10  2023     10     202310   -3.18 -4.05  0.19  0.47  2.47   
295827  12503M10  2023     11     202311    8.83 -0.11  1.66  0.44 -3.81   
295828  12503M10  2023     12     202312    4.87  7.33  4.92  0.43 -3.04   

         CMA  ...  Method_Estimated     GMB_U     GMB_S  IndGMB_U  IndGMB_S  \
0      -

In [3]:
# Get sector dummies!
sectordummies = pd.get_dummies(data['sector'], dtype = int, prefix='sector')
# Concatenate dummy variables with main DataFrame
withsectors = pd.concat([data, sectordummies], axis=1)

# Multiplying by Indicators (REMOVING RETAIL (median))
withsectors['IndU_Acc'] = withsectors['GMB_U'] * withsectors['sector_AccomFood']
withsectors['IndU_Cons'] = withsectors['GMB_U'] * withsectors['sector_Construction']
withsectors['IndU_Fin'] = withsectors['GMB_U'] * withsectors['sector_FinanceIns']
withsectors['IndU_HC'] = withsectors['GMB_U'] * withsectors['sector_Healthcare']
withsectors['IndU_Info'] = withsectors['GMB_U'] * withsectors['sector_Information']
withsectors['IndU_Man'] = withsectors['GMB_U'] * withsectors['sector_Manufacturing']
withsectors['IndU_Oil'] = withsectors['GMB_U'] * withsectors['sector_OilGas']
withsectors['IndU_RE'] = withsectors['GMB_U'] * withsectors['sector_RealEstate']
withsectors['IndU_Serv'] = withsectors['GMB_U'] * withsectors['sector_Services']
withsectors['IndU_Transp'] = withsectors['GMB_U'] * withsectors['sector_TransportWarehouse']
withsectors['IndU_Util'] = withsectors['GMB_U'] * withsectors['sector_Utilities']
withsectors['IndU_Whol'] = withsectors['GMB_U'] * withsectors['sector_Wholesale']

withsectors['IndS_Acc'] = withsectors['GMB_S'] * withsectors['sector_AccomFood']
withsectors['IndS_Cons'] = withsectors['GMB_S'] * withsectors['sector_Construction']
withsectors['IndS_Fin'] = withsectors['GMB_S'] * withsectors['sector_FinanceIns']
withsectors['IndS_HC'] = withsectors['GMB_S'] * withsectors['sector_Healthcare']
withsectors['IndS_Info'] = withsectors['GMB_S'] * withsectors['sector_Information']
withsectors['IndS_Man'] = withsectors['GMB_S'] * withsectors['sector_Manufacturing']
withsectors['IndS_Oil'] = withsectors['GMB_S'] * withsectors['sector_OilGas']
withsectors['IndS_RE'] = withsectors['GMB_S'] * withsectors['sector_RealEstate']
withsectors['IndS_Serv'] = withsectors['GMB_S'] * withsectors['sector_Services']
withsectors['IndS_Transp'] = withsectors['GMB_S'] * withsectors['sector_TransportWarehouse']
withsectors['IndS_Util'] = withsectors['GMB_S'] * withsectors['sector_Utilities']
withsectors['IndS_Whol'] = withsectors['GMB_S'] * withsectors['sector_Wholesale']

In [4]:
# Get time dummies!
yeardummies = pd.get_dummies(data['year'], dtype = int, prefix='year')
# Drop first year (2002) to avoid multicollinearity with time dummies
yeardummies.drop('year_2002', axis=1, inplace=True)
# Concatenate dummy variables with main DataFrame
factordata = pd.concat([withsectors, yeardummies], axis=1)

**(1) FF3 + LogCO2**

In [5]:
X1 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'LogCO2',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
y1 = factordata["ExcessReturn"]
model1 = sm.OLS(y1, X1).fit(cov_type='HC1')
print(model1.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                 1.587e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:52:54   Log-Likelihood:             1.7631e+05
No. Observations:              295829   AIC:                        -3.526e+05
Df Residuals:                  295803   BIC:                        -3.523e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1261      0.002    -54.737      0.0

**(2) FF5 + LogCO2**

In [6]:
X2 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'RMW', 'CMA', 'HML', 'LogCO2',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
y2 = factordata["ExcessReturn"]
model2 = sm.OLS(y2, X2).fit(cov_type='HC1')
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                 1.480e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:52:54   Log-Likelihood:             1.7658e+05
No. Observations:              295829   AIC:                        -3.531e+05
Df Residuals:                  295801   BIC:                        -3.528e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.1218      0.002    -51.668      0.0

**(3) FF3 + CO2 Intensity**

In [7]:
X3 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'CO2_Intensity',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
y3 = factordata["ExcessReturn"]
T5model3 = sm.OLS(y3, X3).fit(cov_type='HC1')
print(T5model3.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.511
Model:                            OLS   Adj. R-squared:                  0.511
Method:                 Least Squares   F-statistic:                 1.579e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:52:55   Log-Likelihood:             1.7631e+05
No. Observations:              295829   AIC:                        -3.526e+05
Df Residuals:                  295803   BIC:                        -3.523e+05
Df Model:                          25                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.1278      0.002    -62.961

**(4) FF5 + CO2 Intensity**

In [8]:
X4 = sm.add_constant(factordata[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'CO2_Intensity',
                                   'year_2003', 'year_2004', 'year_2005', 'year_2006', 'year_2007', 
                                   'year_2008', 'year_2009', 'year_2010', 'year_2011', 'year_2012', 
                                   'year_2013', 'year_2014', 'year_2015', 'year_2016', 'year_2017', 
                                   'year_2018', 'year_2019', 'year_2020', 'year_2021', 'year_2022', 'year_2023']])
y4 = factordata["ExcessReturn"]
model4 = sm.OLS(y4, X4).fit(cov_type='HC1')
print(model4.summary())

                            OLS Regression Results                            
Dep. Variable:           ExcessReturn   R-squared:                       0.512
Model:                            OLS   Adj. R-squared:                  0.512
Method:                 Least Squares   F-statistic:                 1.473e+04
Date:                Wed, 09 Apr 2025   Prob (F-statistic):               0.00
Time:                        22:52:55   Log-Likelihood:             1.7658e+05
No. Observations:              295829   AIC:                        -3.531e+05
Df Residuals:                  295801   BIC:                        -3.528e+05
Df Model:                          27                                         
Covariance Type:                  HC1                                         
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
const            -0.1235      0.002    -59.129

In [9]:
# RRstargazer = Stargazer([T5model1, T5model2, T5model3, T5model4])
# print(RRstargazer.render_latex())